In [ ]:
import pandas as pd
import numpy as np

import scipy.stats as stats
import seaborn as sns
from matplotlib import pyplot as plt

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans

import warnings

In [ ]:
warnings.filterwarnings(action='ignore',category=FutureWarning)
warnings.filterwarnings(action='ignore',category=DeprecationWarning)
warnings.filterwarnings(action='ignore',category=UserWarning)

# Part 1 - Building up a basic predictive model

## Data cleaning and transformation

### Show the shape of the dataset

In [ ]:
pd.set_option('display.max_columns', None)
df = pd.read_csv("Manhattan12.csv", skiprows=4)
df.shape

### Rename incorrectly formatted column names (e.g. SALE\nPRICE)

In [ ]:
df.columns = df.columns.str.replace("\n", "").str.replace(" ", "_")
df.rename(columns={"SALEPRICE": "SALE_PRICE", "APARTMENTNUMBER": "APARTMENT_NUMBER"}, inplace=True)

### Create list of categorical variables and another for the numerical variables
### For each numerical column, remove the ',' the '$' for the sale price, and then convert them to numeric.
### Convert the 'SALE DATE' to datetime.

In [ ]:
df.columns = df.columns.str.strip().str.replace("\n", "")

def format_and_convert_column_to_numeric(col, dtype):
    df[col] = df[col].str.replace(",","").str.replace("$","").astype(dtype)
    
format_and_convert_column_to_numeric('SALE_PRICE', np.int64)

# Convert To datetime
df['SALE_DATE'] =pd.to_datetime(df['SALE_DATE'], dayfirst=True)

df['YEAR_BUILT'] = df['YEAR_BUILT'].astype(np.int64)

### Create list of categorical variables and another for the numerical variables

In [ ]:
categorical = df.select_dtypes(include='object').columns.tolist()
numerical = df.select_dtypes(include=['int', 'float']).columns.tolist()
print("Categorical columns: ", categorical)
print("Numerical columns: ", numerical)

### For each categorical variable, remove the spaces, and then replace the empty string '' by NaN.

In [ ]:
df[categorical] = df[categorical].applymap(lambda x: x.replace(r" +", ""))
df[categorical] = df[categorical].applymap(lambda x: " ".join(x.split()))

### Replace the zeros in Prices, Land squares, etc. by NaN

In [ ]:
df[categorical] = df[categorical].replace("", np.nan)
df[numerical] = df[numerical].replace(0, np.nan)

### Show a summary of all missing values as well as the summary statistics (Percentage)

In [ ]:
df.isna().sum() / df.shape[0] * 100

### Drop the columns 'BOROUGH', 'EASE-MENT', 'APARTMENT NUMBER'

In [ ]:
df.drop(columns=['BOROUGH', 'EASE-MENT', 'APARTMENT_NUMBER'], inplace=True)
df.columns

### Drop duplicates if any
### Drop rows with NaN values

In [ ]:
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)
df.isna().sum()

### Identify and remove outliers if any

In [ ]:
numerical = df.select_dtypes(include=['int', 'float']).columns.tolist()
z_scores = np.abs(stats.zscore(df[numerical]))

In [ ]:
df = df[(z_scores<3).all(axis=1)]

### Show the shape of the resulting dataframe.

In [ ]:
df.shape

In [ ]:
df.describe()

### Consider the log of the prices and normalise the data.

In [ ]:

df['SALE_PRICE'] = np.log(df['SALE_PRICE'])


In [ ]:
numerical.remove("SALE_PRICE")
df[numerical] = (df[numerical] - df[numerical].min()) / (df[numerical].max() - df[numerical].min())
numerical.append("SALE_PRICE")

In [ ]:
df.describe()

## Data Exploration

### Visualise the prices across neighborhood

In [ ]:
plt.figure(figsize=(25, 12))
sns.set_theme(font_scale=1.2)
fig = sns.barplot(data=df ,y='SALE_PRICE', x='NEIGHBORHOOD', order=df.groupby(['NEIGHBORHOOD']).mean().sort_values('SALE_PRICE').index)
plt.xticks(rotation=70)

### Correlation Matrix

In [ ]:
sns.heatmap(df[numerical].corr().round(2), annot=True)

### Scatter Matrix Plot

In [ ]:
sns.set_theme(style='ticks', font_scale=1.2, palette='mako')
sns.pairplot(data=df, hue='SALE_PRICE')

In [ ]:
plt.figure(figsize=(8, 10))
fig = sns.boxplot(data=df, palette='pastel').set(title='Box plot')
sns.set( font_scale=0.8)
plt.tick_params(axis='both', which='major', labelsize=10, rotation=60)

In [ ]:
df.info()

## Model building

### Select the predictors that would have impact in predicting house prices. 

In [ ]:
df = df.iloc[:, [0, 1, 8, 10, 11, 12, 15, 3, 4, 7, 9, 13, 14, 16]]
df.iloc[:, :10]

In [ ]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0, 1, 2, 3, 4, 5, 6, 7, 8])], remainder='passthrough')
X = ct.fit_transform(X.values)

### Split the data into a training and test sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y.values, test_size=0.2, random_state=69420)

### Train Model

In [ ]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)

### Predict the test dataset

In [ ]:
y_pred = regressor.predict(X_test)

### Model evaluation

In [ ]:
mae = mean_absolute_error(y_pred=y_pred, y_true=y_test)
mse = mean_squared_error(y_pred, y_test)
rmse = np.sqrt(mse)

print(f'Mean Absolute Error - {mae}')
print(f'Mean Squared Error - {mse}')
print(f'Root Mean Squared Error - {rmse}')

### Comparing the y_true and y_pred values side-by-side

In [ ]:
predictions = pd.DataFrame(np.concatenate((y_test.reshape(-1, 1), y_pred.reshape(-1, 1)), axis=1), columns=['Y_TRUE', 'Y_PRED'])
predictions

### Cross Validation

In [ ]:
results = cross_validate(LinearRegression(), X_test, y_test, cv=5, scoring=('r2', 'neg_root_mean_squared_error'))
print("Cross Validation results")
results

### Histogram of the residuals

In [ ]:
plt.figure(figsize=(10, 10))
sns.set_theme(style='whitegrid', font_scale=1, palette='Spectral')
sns.histplot(y_test - y_pred, bins=500)

# Part 1 - Building up a basic predictive model

## Data cleaning and transformation

### Show the shape of the dataset

In [ ]:
pd.set_option('display.max_columns', None)
df = pd.read_csv("Manhattan12.csv", skiprows=4)
df.head(5)

In [ ]:
# Dtype conversion
df.columns = df.columns.str.strip().str.replace("\n", "")
    
format_and_convert_column_to_numeric('SALEPRICE', np.int64)
format_and_convert_column_to_numeric('GROSS SQUARE FEET', np.float64)
format_and_convert_column_to_numeric('LAND SQUARE FEET', np.float64)
format_and_convert_column_to_numeric('RESIDENTIAL UNITS', np.int64)
format_and_convert_column_to_numeric('TOTAL UNITS', np.int64)

# Convert To datetime
df['SALE DATE'] =pd.to_datetime(df['SALE DATE'], dayfirst=True)

df['YEAR BUILT'] = df['YEAR BUILT'].astype(np.int64)

In [ ]:
df.columns = df.columns.str.replace("\n", "").str.replace(" ", "_")
df.rename(columns={"SALEPRICE": "SALE_PRICE", "APARTMENTNUMBER": "APARTMENT_NUMBER"}, inplace=True)
df.columns = df.columns.str.lower()

In [ ]:
categorical = df.select_dtypes(include='object').columns.tolist()
numerical = df.select_dtypes(include=['int', 'float']).columns.tolist()

In [ ]:
df[categorical] = df[categorical].apply(lambda x: x.str.strip())
df.replace("", np.nan, inplace=True)
df.replace(0, np.nan, inplace=True)
df.isna().sum() / len(df) * 100

In [ ]:
df.dropna(subset=['year_built'], how='any', inplace=True)

In [ ]:
df['sale_year'] = pd.DatetimeIndex(df.sale_date).year
df['sale_month'] = pd.DatetimeIndex(df.sale_date).month
df['building_age'] = 2013 - df['year_built']

In [ ]:
df.drop(columns=['borough', 'address', 'ease-ment', 'apartment_number', 'sale_date', 'year_built'], inplace=True)


In [ ]:
categorical = df.select_dtypes(include='object').columns.tolist()
numerical = df.select_dtypes(include=['int', 'float']).columns.tolist()
print("Categorical columns: ", categorical)
print("Numerical columns: ", numerical)

In [ ]:
df.describe()

In [ ]:
min_values = df[numerical].min()

In [ ]:
df['sale_price'].describe()

In [ ]:
df = df[(df['sale_price'] > 1000000) & (df['sale_price'] < 15000000)]

In [ ]:
df.describe()

In [ ]:
df.dropna(subset=categorical, how='any', inplace=True)

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.drop(columns=['building_class_at_present', 'tax_class_at_present', 'tax_class_at_time_of_sale', 'building_class_at_time_of_sale', 'total_units'], inplace=True)

In [ ]:
categorical = df.select_dtypes(include='object').columns.tolist()
numerical = df.select_dtypes(include=['int', 'float']).columns.tolist()
print("Categorical columns: ", categorical)
print("Numerical columns: ", numerical)

In [ ]:
plt.figure(figsize=(15, 10))
sns.heatmap(df[numerical].corr().round(2), annot=True, cmap='hot')

In [ ]:
X = df.drop(columns='sale_price')
y = df['sale_price']
X.info()

In [ ]:
neigborhood_le = LabelEncoder()
building_class_category_le = LabelEncoder()
X['encoded_neighborhood'] = neigborhood_le.fit_transform(X['neighborhood'])
X['encoded_building_class_category'] = building_class_category_le.fit_transform(X['building_class_category'])
X.drop(columns=['building_class_category', 'neighborhood'], inplace=True)
final_columns = X.columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_test = pd.DataFrame(X_test, columns=final_columns)
X_train = pd.DataFrame(X_train, columns=final_columns)

In [ ]:
train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [ ]:
final_columns = train.columns

In [ ]:
train.isnull().sum()

In [ ]:
le = LinearRegression()
imputer = IterativeImputer(estimator=le, min_value=train.min())
imputer.fit(train)
train = pd.DataFrame(imputer.transform(train), columns=final_columns)
test = pd.DataFrame(imputer.transform(test), columns=final_columns)

In [ ]:
train[['commercial_units', 'residential_units']] = train[['commercial_units', 'residential_units']].applymap(lambda x: round(x))
test[['commercial_units', 'residential_units']] = test[['commercial_units', 'residential_units']].applymap(lambda x: round(x))

In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
train[['encoded_neighborhood', 'encoded_building_class_category']] = train[['encoded_neighborhood', 'encoded_building_class_category']].astype(np.object)

In [ ]:
plt.figure(figsize=(25, 12))
sns.set_theme(font_scale=1.2)
fig = sns.barplot(data=train ,y='sale_price', x='encoded_neighborhood', order=train.groupby(['encoded_neighborhood']).mean().sort_values('sale_price').index)
plt.xticks(rotation=70)

In [ ]:
train['sale_price'] = np.log(train['sale_price'])
test['sale_price'] = np.log(test['sale_price'])

In [ ]:
plt.figure(figsize=(25, 12))
sns.set_theme(font_scale=1.2)
fig = sns.barplot(data=train ,y='sale_price', x='encoded_neighborhood', order=train.groupby(['encoded_neighborhood']).mean().sort_values('sale_price').index)
plt.xticks(rotation=70)

In [ ]:
neigborhood_le.classes_

In [ ]:
plt.figure(figsize=(25, 12))
sns.set_theme(font_scale=1.2)
sns.heatmap(train[numerical].corr().round(2), annot=True)

In [ ]:
train[['encoded_neighborhood', 'encoded_building_class_category']] = train[['encoded_neighborhood', 'encoded_building_class_category']].astype(np.int)

In [ ]:
Q1 = train.quantile(0.25)
Q3 = train.quantile(0.75)
IQR = Q3 - Q1
train = train[~((train < (Q1 - 1.5 * IQR)) |(train > (Q3 + 1.5 * IQR))).any(axis=1)]
'''
Zolduoarrati, E. (2019, December 17). Remove Outliers in Pandas DataFrame using Percentiles. Stack Overflow. https://stackoverflow.com/questions/35827863/remove-outliers-in-pandas-dataframe-using-percentiles
'''

In [ ]:
numerical.extend(['encoded_neighborhood', 'encoded_building_class_category'])

In [ ]:

standard_scaler = StandardScaler()
X_columns = X_train.columns

In [ ]:
X_train_sc = standard_scaler.fit_transform(train.iloc[:, :-1])
X_train_sc

In [ ]:
X_train = pd.DataFrame(X_train_sc, columns=X_columns)

In [ ]:
X_test_sc = standard_scaler.transform(test.iloc[:, :-1])
X_test = pd.DataFrame(X_test_sc, columns=X_columns)

In [ ]:
sns.set_style("whitegrid")
sns.set_palette("husl")
sns.set_context("notebook")

wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=500, random_state=69420)
    kmeans.fit(X_train.values)
    wcss.append(kmeans.inertia_)

sns.lineplot(x=range(1, 11), y=wcss, marker='o')
plt.title('Elbow Method')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
number_of_clusters = 2

k_means_model = KMeans(n_clusters=number_of_clusters, random_state=69420, max_iter=500)

In [ ]:
k_means_model.fit(X_train)

In [ ]:
train_clusters = k_means_model.predict(X_train)

In [ ]:
labels = k_means_model.labels_

In [ ]:
sns.scatterplot(x=X_train['land_square_feet'], y=X_train['gross_square_feet'], hue=k_means_model.labels_, palette='hot')

# add cluster centroids to the scatter plot
sns.scatterplot(x=k_means_model.cluster_centers_[:, 5], y=k_means_model.cluster_centers_[:, 6], color='black', marker='X', s=100)

plt.title('Clusters and Centroids')
plt.xlabel('Land Square Feet')
plt.ylabel('Gross Square Feet')
plt.show()

In [ ]:
X_train_a = train[train_clusters == 0].iloc[:, :-1]
y_train_a = train[train_clusters == 0].iloc[:, -1]
X_train_b = train[train_clusters == 1].iloc[:, :-1]
y_train_b = train[train_clusters == 1].iloc[:, -1]

In [ ]:
regressor_a = LinearRegression()
regressor_b = LinearRegression()

regressor_a.fit(X_train_a, y_train_a)
regressor_b.fit(X_train_b, y_train_b)

In [ ]:
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]
test_clusters = k_means_model.predict(X_test)

In [ ]:
X_test_a = X_test[test_clusters == 0].values
y_test_a = y_test[test_clusters == 0].values
X_test_b = X_test[test_clusters == 1].values
y_test_b = y_test[test_clusters == 1].values

In [ ]:
y_pred_a = regressor_a.predict(X_test_a)
y_pred_b = regressor_b.predict(X_test_b)

In [ ]:

cluster_a_pred = pd.DataFrame(np.concatenate((y_test_a.reshape(-1, 1), y_pred_a.reshape(-1, 1)), axis=1), columns=['Y_TRUE', 'Y_PRED'])
cluster_b_pred = pd.DataFrame(np.concatenate((y_test_b.reshape(-1, 1), y_pred_b.reshape(-1, 1)), axis=1), columns=['Y_TRUE', 'Y_PRED'])
final_pred = pd.concat([cluster_a_pred, cluster_b_pred], axis=0)

In [ ]:
mae = mean_absolute_error(final_pred.iloc[:, 0], final_pred.iloc[:, 1])
mse = mean_squared_error(final_pred.iloc[:, 0], final_pred.iloc[:, 1])
rmse = np.sqrt(mse)

print(f'Mean Absolute Error - {mae}')
print(f'Mean Squared Error - {mse}')
print(f'Root Mean Squared Error - {rmse}')